In [1]:
import csv
import io
import os
import time
import uuid 
import re
import pandas as pd
from langchain_chroma import Chroma
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_docling import DoclingLoader
from langchain_docling.loader import ExportType
from langchain_text_splitters import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter

In [2]:
df = pd.read_csv('AWSDocs.csv')

In [3]:
CHROMA_DB_PATH = "./chroma_db_AWSDocs"
EMBEDDING_MODEL_NAME = "nomic-embed-text" 
BATCH_SIZE = 50  
CHUNK_SIZE = 1000 
CHUNK_OVERLAP = 200

In [4]:
embeddings = OllamaEmbeddings(model=EMBEDDING_MODEL_NAME)

In [5]:
vector_store = Chroma(
    collection_name="AWSDocs",
    persist_directory=CHROMA_DB_PATH,
    embedding_function=embeddings
)

2025-11-12 19:42:38,442 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


In [8]:
import httpx # Make sure this is imported
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma

# Re-create your embeddings object with a timeout
embeddings = OllamaEmbeddings(
    model="nomic-embed-text",
    timeout=60.0 # <-- ADD THIS: Fail after 30 seconds
)

# Re-create your vector store to use the new embedding function
vector_store = Chroma(
    persist_directory=CHROMA_DB_PATH,
    embedding_function=embeddings
)

print("Embeddings and vector store re-initialized with a 30-second timeout.")

ValidationError: 1 validation error for OllamaEmbeddings
timeout
  Extra inputs are not permitted [type=extra_forbidden, input_value=60.0, input_type=float]
    For further information visit https://errors.pydantic.dev/2.12/v/extra_forbidden

In [9]:
embeddings = OllamaEmbeddings(
    model="nomic-embed-text",
    timeout=60.0 # <-- ADD THIS: Fail after 30 seconds
)

ValidationError: 1 validation error for OllamaEmbeddings
timeout
  Extra inputs are not permitted [type=extra_forbidden, input_value=60.0, input_type=float]
    For further information visit https://errors.pydantic.dev/2.12/v/extra_forbidden

In [6]:
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on)

In [7]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP
)

In [8]:
# loader = DoclingLoader(
#     file_path='https://docs.aws.amazon.com/pdfs/next-generation-sagemaker/latest/userguide/next-generation-sagemaker-ug.pdf',
#     export_type=ExportType.MARKDOWN,
# )

In [9]:
for row in df.iterrows():
    url = row[1]['PDF_URL']
    domain = row[1]['Domain']
    service = row[1]['Service']
    if not url:
        continue
    else:
        try:
            loader = DoclingLoader(
                file_path=url,
                export_type=ExportType.MARKDOWN,
            )
            docs_as_markdown = loader.load()
            markdown_content = docs_as_markdown[0].page_content
            semantic_chunks = markdown_splitter.split_text(markdown_content)

            for chunk in semantic_chunks:
                chunk.metadata["domain"] = domain
                chunk.metadata["service"] = service
                chunk.metadata["source"] = url

            final_chunks = text_splitter.split_documents(semantic_chunks)

            if final_chunks:
                print(f"Found {len(final_chunks)} final chunks to add.") 

            chunk_ids = [str(uuid.uuid4()) for _ in final_chunks]

            for i in range(0, len(final_chunks)):
        
                # Get the current chunk
                chunk = final_chunks[i]
                
                h1 = chunk.metadata.get("Header 1", "")
                h2 = chunk.metadata.get("Header 2", "")
                h3 = chunk.metadata.get("Header 3", "")
                
                is_toc = 'table of contents' in h1.lower() or \
                        'table of contents' in h2.lower() or \
                        'table of contents' in h3.lower()

                content_to_check = chunk.page_content.strip() # Remove leading/trailing spaces
                is_table_junk = False
                if content_to_check: # Only check if the string is not empty
                    is_table_junk = bool(re.fullmatch(r'[|\-\s]+', content_to_check))
                
                
                if is_toc or is_table_junk:
                    if is_toc:
                        print(f"Skipping chunk {i+1} (Table of Contents)")
                    if is_table_junk:
                        print(f"Skipping chunk {i+1} (Markdown table fragment)")
                    pass # Skip this chunk
                
                else:
                    batch_docs = [chunk] # Get the chunk as a list
                    batch_ids = [chunk_ids[i]] # Get the corresponding ID
                    
                    batch_num = i + 1
                    total_batches = len(final_chunks)

                    print(f"  Adding batch {batch_num}/{total_batches}...")
                    
                    try:
                        vector_store.add_documents(
                            documents=batch_docs,
                            ids=batch_ids
                        )
                    except Exception as e:
                        print(f"Error adding batch {batch_num}/{total_batches}: {e}")
                        continue 

        except Exception as e:
            print(f"Error processing {url}: {e}")
            continue


2025-11-12 09:46:03,873 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-11-12 09:46:04,165 - INFO - Going to convert document batch...
2025-11-12 09:46:04,166 - INFO - Initializing pipeline for StandardPdfPipeline with options hash 44ae89a68fc272bc7889292e9b5a1bad
2025-11-12 09:46:04,184 - INFO - Loading plugin 'docling_defaults'
2025-11-12 09:46:04,187 - WARNING - The plugin langchain_docling will not be loaded because Docling is being executed with allow_external_plugins=false.
2025-11-12 09:46:04,187 - INFO - Registered picture descriptions: ['vlm', 'api']
2025-11-12 09:46:04,193 - INFO - Loading plugin 'docling_defaults'
2025-11-12 09:46:04,197 - WARNING - The plugin langchain_docling will not be loaded because Docling is being executed with allow_external_plugins=false.
2025-11-12 09:46:04,197 - INFO - Registered ocr engines: ['auto', 'easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']
2025-11-12 09:46:05,600 - INFO - Auto OCR model selected ocrmac.
2025-11-12 09

Found 10887 final chunks to add.
  Adding batch 1/10887...


2025-11-12 10:03:41,309 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-11-12 10:03:41,414 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-11-12 10:03:41,458 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-11-12 10:03:41,517 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-11-12 10:03:41,545 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-11-12 10:03:41,570 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"


  Adding batch 2/10887...
  Adding batch 3/10887...
Skipping chunk 4 (Table of Contents)
Skipping chunk 5 (Table of Contents)
Skipping chunk 6 (Table of Contents)
Skipping chunk 7 (Table of Contents)
Skipping chunk 8 (Table of Contents)
Skipping chunk 9 (Table of Contents)
Skipping chunk 10 (Table of Contents)
Skipping chunk 11 (Table of Contents)
Skipping chunk 12 (Table of Contents)
Skipping chunk 13 (Table of Contents)
Skipping chunk 14 (Table of Contents)
Skipping chunk 15 (Table of Contents)
Skipping chunk 16 (Table of Contents)
Skipping chunk 17 (Table of Contents)
Skipping chunk 18 (Table of Contents)
Skipping chunk 19 (Table of Contents)
Skipping chunk 20 (Table of Contents)
Skipping chunk 21 (Table of Contents)
Skipping chunk 22 (Table of Contents)
Skipping chunk 23 (Table of Contents)
Skipping chunk 24 (Table of Contents)
Skipping chunk 25 (Table of Contents)
Skipping chunk 26 (Table of Contents)
Skipping chunk 27 (Table of Contents)
Skipping chunk 28 (Table of Contents)
Skip

2025-11-12 10:03:41,603 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-11-12 10:03:41,629 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-11-12 10:03:41,656 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-11-12 10:03:41,683 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-11-12 10:03:41,709 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-11-12 10:03:41,751 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-11-12 10:03:41,776 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-11-12 10:03:41,801 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"


...

  Adding batch 10882/10887...
  Adding batch 10883/10887...
  Adding batch 10884/10887...
  Adding batch 10885/10887...


2025-11-12 10:15:08,225 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-11-12 10:15:08,269 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-11-12 10:15:08,316 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"


  Adding batch 10886/10887...
  Adding batch 10887/10887...


2025-11-12 10:15:11,144 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-11-12 10:15:11,280 - INFO - Going to convert document batch...
2025-11-12 10:15:11,280 - INFO - Initializing pipeline for StandardPdfPipeline with options hash 44ae89a68fc272bc7889292e9b5a1bad
2025-11-12 10:15:11,281 - INFO - Auto OCR model selected ocrmac.
2025-11-12 10:15:11,282 - INFO - Accelerator device: 'mps'
2025-11-12 10:15:12,616 - INFO - Accelerator device: 'mps'
2025-11-12 10:15:13,277 - INFO - Processing document lambda-dg.pdf


KeyboardInterrupt: 